# Environment Settings

In [55]:
import torch
print(torch.version.cuda)


12.4


In [56]:
import torch

if torch.cuda.is_available():
    gpu_id = torch.cuda.current_device()
    gpu_name = torch.cuda.get_device_name(gpu_id)
    gpu_capability = torch.cuda.get_device_capability(gpu_id)
    total_memory = torch.cuda.get_device_properties(gpu_id).total_memory
    
    print(f"Total number of GPU: {torch.cuda.device_count()}")  # Number of GPUs available
    print(f"Total GPU memory: {total_memory / 1e9} GB")
    print(f"GPU ID: {gpu_id}")
    print(f"GPU Name: {gpu_name}")
    print(f"GPU Compute Capability: {gpu_capability}")
else:
    print("No GPU is available.")

Total number of GPU: 4
Total GPU memory: 51.033931776 GB
GPU ID: 0
GPU Name: NVIDIA RTX A6000
GPU Compute Capability: (8, 6)


In [57]:
import torch

# 강제로 CUDA 디바이스 설정
torch.cuda.set_device(0)

# 확인
print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


Current Device: 0
Device Name: NVIDIA RTX A6000


device(type='cuda')

In [58]:
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [59]:
import pandas as pd
import numpy as np

In [60]:
data = pd.read_csv('../dataset/exchange_rate.csv', index_col='date', parse_dates=True)

In [61]:
data.head()

,0,1,2,3,4,5,6,OT
date,,,,,,,,
1990-01-01,0.7855,1.6110,0.861698,0.634196,0.211242,0.006838,0.525486,0.5930
1990-01-02,0.7818,1.6100,0.861104,0.633513,0.211242,0.006863,0.523972,0.5940
1990-01-03,0.7867,1.6293,0.861030,0.648508,0.211242,0.006975,0.526316,0.5973
1990-01-04,0.7860,1.6370,0.862069,0.650618,0.211242,0.006953,0.523834,0.5970
1990-01-05,0.7849,1.6530,0.861995,0.656254,0.211242,0.006940,0.527426,0.5985


In [62]:
print(data.dtypes)
print('shape: ', data.shape)

0     float64
1     float64
2     float64
3     float64
4     float64
5     float64
6     float64
OT    float64
dtype: object
shape:  (7588, 8)


In [63]:
target = data['OT']
features = data.drop(['OT'],axis=1)

In [64]:
features['DATE'] = features.index.strftime('%Y%m%d%H').astype(int)

In [65]:
target.index

DatetimeIndex(['1990-01-01', '1990-01-02', '1990-01-03', '1990-01-04',
               '1990-01-05', '1990-01-06', '1990-01-07', '1990-01-08',
               '1990-01-09', '1990-01-10',
               ...
               '2010-10-01', '2010-10-02', '2010-10-03', '2010-10-04',
               '2010-10-05', '2010-10-06', '2010-10-07', '2010-10-08',
               '2010-10-09', '2010-10-10'],
              dtype='datetime64[ns]', name='date', length=7588, freq=None)

In [66]:
import torch
import numpy as np
import random

def set_seed(seed=42):
    random.seed(seed)                           # Python random
    np.random.seed(seed)                        # NumPy
    torch.manual_seed(seed)                     # PyTorch CPU
    torch.cuda.manual_seed(seed)                # PyTorch GPU (single)
    torch.cuda.manual_seed_all(seed)            # PyTorch GPU (multi)
    torch.backends.cudnn.deterministic = True   # 연산 순서 고정
    torch.backends.cudnn.benchmark = False      # 성능 대신 일관성 선택

In [67]:
set_seed(24) 

# Model Modification for Linear (Look-ahead Augmentation)

In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Model(nn.Module):
    """
    Linear model with prediction-based augmentation (attach_pv)
    """
    def __init__(self, configs):
        super(Model, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.channels = configs.enc_in

        # --- 새 옵션: attach_pv ---
        self.attach_pv = getattr(configs, "attach_pv", False)

        # 추가 시퀀스 길이 = pred_len // 3 (DLinear 개선 버전과 동일)
        self.extra_len = self.pred_len // 3

        # augmented seq_len = 기본 seq_len + extra_len (if attach_pv)
        self.seq_len_aug = self.seq_len + (self.extra_len if self.attach_pv else 0)

        self.individual = configs.individual

        # ----- Linear layer(s) -----
        if self.individual:
            self.Linear = nn.ModuleList()
            for i in range(self.channels):
                self.Linear.append(nn.Linear(self.seq_len_aug, self.pred_len))
        else:
            self.Linear = nn.Linear(self.seq_len_aug, self.pred_len)


    def forward(self, x, ground_truth=None):
        """
        x: [B, L, C]
        ground_truth: [B, L_gt, C] or None
        """
        B, L, C = x.size()

        # --------------------------------------------------
        # 1) 예측값 기반 증강: attach_pv = True & ground_truth 제공
        # --------------------------------------------------
        if self.attach_pv and (ground_truth is not None):
            # ground truth의 앞 extra_len 만큼만 사용 (DLinear 개선 버전과 동일)
            gt_part = ground_truth[:, :self.extra_len, :]  # [B, extra_len, C]

            # concat → [B, L + extra_len, C]
            x = torch.cat([x, gt_part], dim=1)

        # --------------------------------------------------
        # 2) Linear projection (기존 Linear.py 구조 동일)
        # --------------------------------------------------
        if self.individual:
            out = torch.zeros([B, self.pred_len, C], dtype=x.dtype).to(x.device)
            for i in range(C):
                out[:, :, i] = self.Linear[i](x[:, :, i])
            return out

        else:
            # shared linear: (B, C, L_aug) → Linear → (B, C, pred_len)
            out = self.Linear(x.permute(0, 2, 1))
            return out.permute(0, 2, 1)


# Split the Data

In [69]:
import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# 0. 데이터 불러오기
target = data['OT'].values.reshape(-1, 1)  # 단변량 시계열

# 1. 입력/출력 길이 정의
seq_len  = 336
pred_len = 96

# 2. 비율 기반 분할 (Dataset_Custom 기준)
total_len  = len(target)
num_train  = int(total_len * 0.7)
num_test   = int(total_len * 0.2)
num_val    = total_len - num_train - num_test

border1s = [
    0,
    num_train - seq_len,
    total_len - num_test - seq_len
]
border2s = [
    num_train,
    num_train + num_val,
    total_len
]

# 3. 구간 나누기
train = target[border1s[0]:border2s[0]]
val   = target[border1s[1]:border2s[1]]
test  = target[border1s[2]:border2s[2]]

# 4. 표준화 (train 기준)
scaler = StandardScaler()
train = scaler.fit_transform(train)
val   = scaler.transform(val)
test  = scaler.transform(test)

# 5. 시퀀스 생성 함수 (단변량)
def create_inout_sequences_univariate(data, seq_len, pred_len):
    seqs = []
    for i in range(len(data) - seq_len - pred_len + 1):
        seq_x = data[i : i + seq_len]
        seq_y = data[i + seq_len : i + seq_len + pred_len]
        seqs.append((seq_x, seq_y))
    return seqs

train_data = create_inout_sequences_univariate(train, seq_len, pred_len)
val_data   = create_inout_sequences_univariate(val,   seq_len, pred_len)
test_data  = create_inout_sequences_univariate(test,  seq_len, pred_len)

# 6. 텐서 변환
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_sequences = torch.tensor([x[0] for x in train_data]).float().to(device)
train_labels    = torch.tensor([x[1] for x in train_data]).float().to(device)

val_sequences   = torch.tensor([x[0] for x in val_data]).float().to(device)
val_labels      = torch.tensor([x[1] for x in val_data]).float().to(device)

test_sequences  = torch.tensor([x[0] for x in test_data]).float().to(device)
test_labels     = torch.tensor([x[1] for x in test_data]).float().to(device)

# 7. 확인
print("Train Sequences:", train_sequences.shape)
print("Train Labels:   ", train_labels.shape)
print("Val Sequences:  ", val_sequences.shape)
print("Val Labels:     ", val_labels.shape)
print("Test Sequences: ", test_sequences.shape)
print("Test Labels:    ", test_labels.shape)


Train Sequences: torch.Size([4880, 336, 1])
Train Labels:    torch.Size([4880, 96, 1])
Val Sequences:   torch.Size([665, 336, 1])
Val Labels:      torch.Size([665, 96, 1])
Test Sequences:  torch.Size([1422, 336, 1])
Test Labels:     torch.Size([1422, 96, 1])


In [70]:
# DataLoader 설정
batch_size = 8

# Train DataLoader
train_dataset = TensorDataset(train_sequences, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Validation DataLoader
val_dataset = TensorDataset(val_sequences, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Test DataLoader
test_dataset = TensorDataset(test_sequences, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 1st Model Training (Early Stopping)

In [71]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [72]:
import torch.optim as optim
import os

# Assuming that data is already scaled before being passed to the DataLoader
seq_len = 336
pred_len = 96  # Set pred_len to the desired prediction horizon

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual, learning_rate, lradj, patience, save_path, attach_pv):
        self.seq_len = seq_len
        self.pred_len = pred_len  # Set pred_len appropriately
        self.enc_in = enc_in
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_pv = attach_pv

configs = Configs(seq_len=seq_len, pred_len=pred_len, enc_in=1, individual=False,
                   learning_rate=0.0005, lradj='type1',  patience=3, save_path="./model_ER", attach_pv=False) # 원하는 값으로 설정


model = Model(configs).to(device)
loss_function = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)

if not os.path.exists(configs.save_path):
    os.makedirs(configs.save_path)

# EarlyStopping 클래스 정의
class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), os.path.join(path, 'checkpoint_ES.pth'))
        self.val_loss_min = val_loss



# Dynamic learning rate adjustment function
def adjust_learning_rate(optimizer, epoch, args):
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    elif args.lradj == '3':
        lr_adjust = {epoch: args.learning_rate if epoch < 10 else args.learning_rate*0.1}
    elif args.lradj == '4':
        lr_adjust = {epoch: args.learning_rate if epoch < 15 else args.learning_rate*0.1}
    elif args.lradj == '5':
        lr_adjust = {epoch: args.learning_rate if epoch < 25 else args.learning_rate*0.1}
    elif args.lradj == '6':
        lr_adjust = {epoch: args.learning_rate if epoch < 5 else args.learning_rate*0.1}  

    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print(f'Updating learning rate to {lr}')


# Training loop using DataLoader
def train_model(model, dataloader, val_dataloader, optimizer, loss_function, epochs, pred_len, args):
    early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    for epoch in range(epochs):
        total_loss = 0
        model.train()
        
        # Adjust learning rate
        adjust_learning_rate(optimizer, epoch, args)
        
        for batch_sequences, batch_labels in dataloader:
            optimizer.zero_grad()

            # Forward pass
            output = model(batch_sequences)
            
            # Focus on the prediction horizon for loss calculation
            # Ensure output and batch_labels have the same length (pred_len)
            output = output[:, -pred_len:, :]  # Slice to only use the prediction length
            batch_labels = batch_labels[:, -pred_len:, :]  # Adjust labels to match
            
            # Ensure shapes match before calculating loss
            assert output.shape == batch_labels.shape, f"Output shape {output.shape} and batch_labels shape {batch_labels.shape} must match"
            
            loss = loss_function(output, batch_labels)  # Compute loss
            
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            total_loss += loss.item()

        # Validation step
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for val_batch_sequences, val_batch_labels in val_dataloader:
                val_output = model(val_batch_sequences)
                val_output = val_output[:, -args.pred_len:, :]
                val_batch_labels = val_batch_labels[:, -args.pred_len:, :]
                
                val_loss += loss_function(val_output, val_batch_labels).item()
        
        val_loss /= len(val_dataloader)

        # Log epoch loss
        print(f'Epoch {epoch} | Train Loss: {total_loss / len(dataloader)} | Val Loss: {val_loss}')
        
        # Early Stopping check
        early_stopping(val_loss, model, args.save_path)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break

# Run training
train_model(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=10, pred_len=configs.pred_len, args=configs)




Updating learning rate to 0.001
Epoch 0 | Train Loss: 0.15261425166772527 | Val Loss: 0.3597180667732443
Validation loss decreased (inf --> 0.359718).  Saving model ...
Updating learning rate to 0.0005
Epoch 1 | Train Loss: 0.11488621577193014 | Val Loss: 0.2180175588776668
Validation loss decreased (0.359718 --> 0.218018).  Saving model ...
Updating learning rate to 0.00025
Epoch 2 | Train Loss: 0.11183011307854389 | Val Loss: 0.3296408844845636
EarlyStopping counter: 1 out of 3
Updating learning rate to 0.000125
Epoch 3 | Train Loss: 0.1080545814830016 | Val Loss: 0.3142980486598043
EarlyStopping counter: 2 out of 3
Updating learning rate to 6.25e-05
Epoch 4 | Train Loss: 0.10638638578477454 | Val Loss: 0.2644095815984266
EarlyStopping counter: 3 out of 3
Early stopping triggered. Stopping training.


# 1st Model Training (1 Epoch)

In [73]:
import torch.optim as optim
import os


# Assuming that data is already scaled before being passed to the DataLoader
seq_len = 336
pred_len = 96  # Set pred_len to the desired prediction horizon

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual, learning_rate, lradj, patience, save_path, attach_pv):
        self.seq_len = seq_len
        self.pred_len = pred_len  # Set pred_len appropriately
        self.enc_in = enc_in
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_pv = attach_pv

configs = Configs(seq_len=seq_len, pred_len=pred_len, enc_in=1, individual=False,
                   learning_rate=0.0005, lradj='type1',  patience=3, save_path="./model_ER", attach_pv=False) # 원하는 값으로 설정


model = Model(configs).to(device)
loss_function = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)

if not os.path.exists(configs.save_path):
    os.makedirs(configs.save_path)

# Dynamic learning rate adjustment function
def adjust_learning_rate(optimizer, epoch, args):
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    elif args.lradj == '3':
        lr_adjust = {epoch: args.learning_rate if epoch < 10 else args.learning_rate*0.1}
    elif args.lradj == '4':
        lr_adjust = {epoch: args.learning_rate if epoch < 15 else args.learning_rate*0.1}
    elif args.lradj == '5':
        lr_adjust = {epoch: args.learning_rate if epoch < 25 else args.learning_rate*0.1}
    elif args.lradj == '6':
        lr_adjust = {epoch: args.learning_rate if epoch < 5 else args.learning_rate*0.1}  

    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print(f'Updating learning rate to {lr}')


# Training loop using DataLoader
def train_model(model, dataloader, val_dataloader, optimizer, loss_function, epochs, pred_len, args):
    # early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    for epoch in range(epochs):

        if epoch == 1:
            torch.save(model.state_dict(), os.path.join(args.save_path, 'checkpoint_1.pth'))
            print("Model saved at epoch 1")

        total_loss = 0
        model.train()
        
        # Adjust learning rate
        adjust_learning_rate(optimizer, epoch, args)
        
        for batch_sequences, batch_labels in dataloader:
            optimizer.zero_grad()

            # Forward pass
            output = model(batch_sequences)
            
            # Focus on the prediction horizon for loss calculation
            # Ensure output and batch_labels have the same length (pred_len)
            output = output[:, -pred_len:, :]  # Slice to only use the prediction length
            batch_labels = batch_labels[:, -pred_len:, :]  # Adjust labels to match
            
            # Ensure shapes match before calculating loss
            assert output.shape == batch_labels.shape, f"Output shape {output.shape} and batch_labels shape {batch_labels.shape} must match"
            
            loss = loss_function(output, batch_labels)  # Compute loss
            
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            total_loss += loss.item()

        # Validation step
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for val_batch_sequences, val_batch_labels in val_dataloader:
                val_output = model(val_batch_sequences)
                val_output = val_output[:, -args.pred_len:, :]
                val_batch_labels = val_batch_labels[:, -args.pred_len:, :]
                
                val_loss += loss_function(val_output, val_batch_labels).item()
        
        val_loss /= len(val_dataloader)

        # Log epoch loss
        print(f'Epoch {epoch} | Train Loss: {total_loss / len(dataloader)} | Val Loss: {val_loss}')

    torch.save(model.state_dict(), os.path.join(args.save_path, f'checkpoint_{epochs}.pth'))
    print(f"Model saved at epoch {epochs}")
        
        # # Early Stopping check
        # early_stopping(val_loss, model, args.save_path)
        # if early_stopping.early_stop:
        #     print("Early stopping triggered. Stopping training.")
        #     break

# Run training
train_model(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=1, pred_len=configs.pred_len, args=configs)




Updating learning rate to 0.001
Epoch 0 | Train Loss: 0.15705875295718186 | Val Loss: 0.21789498651577605
Model saved at epoch 1


# Perform Inference and Form Predicted Value (PV) Sequences

In [21]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch

# 모델 체크포인트가 저장된 epoch 리스트
checkpoint_epochs = ['ES', 1]

# 결과 저장용 딕셔너리
test_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    # 모델 불러오기
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in test_dataloader:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    # 배치 전체 결합
    predictions = np.concatenate(predictions, axis=0)  # shape: (N, pred_len, 1)
    actuals = np.concatenate(actuals, axis=0)

    # 플랫화
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    # 지표 계산
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    # test prediction을 torch.tensor로 변환해서 저장
    num_samples = len(predictions_flatten) // configs.pred_len
    test_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    # 딕셔너리에 저장
    test_1st_results[ckpt_epoch] = {
        f'mse_test_E{ckpt_epoch}': mse,
        f'mae_test_E{ckpt_epoch}': mae,
        f'test_pv_E{ckpt_epoch}': test_pv
    }

# 필요 시 전체 결과를 저장하거나 후처리 가능
# 예: torch.save(all_results, 'all_model_predictions.pt')



[Predicting with model from epoch ES]
Epoch ES | MSE: 0.11179079860448837 | MAE: 0.2555276155471802

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.12221892178058624 | MAE: 0.26797738671302795


In [22]:
test_1st_results.keys()

dict_keys(['ES', 1])

In [23]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch

# 모델 체크포인트가 저장된 epoch 리스트
checkpoint_epochs = ['ES', 1]

# 결과 저장용 딕셔너리
val_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    # 모델 불러오기
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in val_dataloader:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    # 배치 전체 결합
    predictions = np.concatenate(predictions, axis=0)  # shape: (N, pred_len, 1)
    actuals = np.concatenate(actuals, axis=0)

    # 플랫화
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    # 지표 계산
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    # test prediction을 torch.tensor로 변환해서 저장
    num_samples = len(predictions_flatten) // configs.pred_len
    val_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    # 딕셔너리에 저장
    val_1st_results[ckpt_epoch] = {
        f'mse_val_E{ckpt_epoch}': mse,
        f'mae_val_E{ckpt_epoch}': mae,
        f'val_pv_E{ckpt_epoch}': val_pv
    }




[Predicting with model from epoch ES]
Epoch ES | MSE: 0.21841749548912048 | MAE: 0.36545875668525696

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.217507466673851 | MAE: 0.36675718426704407


In [24]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch

# Train DataLoader for inference
train_dataloader_infer = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# 모델 체크포인트가 저장된 epoch 리스트
checkpoint_epochs = ['ES', 1]

# 결과 저장용 딕셔너리
train_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    # 모델 불러오기
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in train_dataloader_infer:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    # 배치 전체 결합
    predictions = np.concatenate(predictions, axis=0)  # shape: (N, pred_len, 1)
    actuals = np.concatenate(actuals, axis=0)

    # 플랫화
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    # 지표 계산
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    # test prediction을 torch.tensor로 변환해서 저장
    num_samples = len(predictions_flatten) // configs.pred_len
    train_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    # 딕셔너리에 저장
    train_1st_results[ckpt_epoch] = {
        f'mse_train_E{ckpt_epoch}': mse,
        f'mae_train_E{ckpt_epoch}': mae,
        f'train_pv_E{ckpt_epoch}': train_pv
    }




[Predicting with model from epoch ES]
Epoch ES | MSE: 0.11328338086605072 | MAE: 0.23653222620487213

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.12368978559970856 | MAE: 0.2478981614112854


In [25]:
print("train_sequences shape: ",train_sequences.shape)
print("val_sequences shape: ",val_sequences.shape)
print("test_sequences shape: ",test_sequences.shape)

print('---------------------------------------------')

print("train_pv shape: ",train_pv.shape)
print("val_pv shape: ",val_pv.shape)
print("test_pv shape: ",test_pv.shape)


print('---------------------------------------------')

print("train_labels shape: ",train_labels.shape)
print("val_labels shape: ",val_labels.shape)
print("test_labels shape: ",test_labels.shape)


train_sequences shape:  torch.Size([4880, 336, 1])
val_sequences shape:  torch.Size([665, 336, 1])
test_sequences shape:  torch.Size([1422, 336, 1])
---------------------------------------------
train_pv shape:  torch.Size([4880, 96, 1])
val_pv shape:  torch.Size([665, 96, 1])
test_pv shape:  torch.Size([1422, 96, 1])
---------------------------------------------
train_labels shape:  torch.Size([4880, 96, 1])
val_labels shape:  torch.Size([665, 96, 1])
test_labels shape:  torch.Size([1422, 96, 1])


# Segment Generation Function (sliding window-based)

In [26]:
def create_sliding_segments(predictions, labels, segment_length, stride=1):
    """
    시퀀스에 슬라이딩 윈도우를 stride 간격으로 적용하여
    (B, num_segments, segment_length, C) 형태로 반환합니다.

    Args:
        predictions: Tensor of shape (B, T, C)
        labels: Tensor of shape (B, T, C)
        segment_length: int, 각 세그먼트의 길이 (예: pred_len // 3)
        stride: int, 슬라이딩 윈도우 stride 간격

    Returns:
        pred_segments: (B, num_segments, segment_length, C)
        label_segments: (B, num_segments, segment_length, C)
    """
    B, T, C = predictions.shape
    num_segments = (T - segment_length) // stride + 1

    pred_segments = []
    label_segments = []

    for i in range(0, T - segment_length + 1, stride):
        pred_seg = predictions[:, i:i+segment_length, :]  # (B, segment_length, C)
        label_seg = labels[:, i:i+segment_length, :]      # (B, segment_length, C)
        pred_segments.append(pred_seg.unsqueeze(1))       # (B, 1, segment_length, C)
        label_segments.append(label_seg.unsqueeze(1))

    pred_segments = torch.cat(pred_segments, dim=1)       # (B, num_segments, segment_length, C)
    label_segments = torch.cat(label_segments, dim=1)     # (B, num_segments, segment_length, C)

    return pred_segments, label_segments

# Segmentation for PV Sequence

In [27]:
configs.pred_len # pred_len // n 을 통해 덧붙일 세그먼트의 길이를 설정

96

In [28]:
segment_len = configs.pred_len // 3
stride = 1  

train_segments_by_epoch = {}
val_segments_by_epoch = {}
test_segments_by_epoch = {}

for epoch in checkpoint_epochs:
    train_pv = train_1st_results[epoch][f'train_pv_E{epoch}']  # (B, T)
    val_pv = val_1st_results[epoch][f'val_pv_E{epoch}']        # (B, T)
    test_pv = test_1st_results[epoch][f'test_pv_E{epoch}']     # (B, T)

    # 라벨은 동일한 GT를 사용
    train_pred_segments, train_label_segments = create_sliding_segments(train_pv, train_labels, segment_len,stride)
    val_pred_segments, val_label_segments = create_sliding_segments(val_pv, val_labels, segment_len, stride)
    test_pred_segments, test_label_segments = create_sliding_segments(test_pv, test_labels, segment_len, stride)

    train_segments_by_epoch[epoch] = {
        f'train_pred_segments_E{epoch}': train_pred_segments,  # shape: (B, 65, segment_len, 1)
        f'train_label_segments_E{epoch}': train_label_segments
    }

    val_segments_by_epoch[epoch] = {
        f'val_pred_segments_E{epoch}': val_pred_segments,
        f'val_label_segments_E{epoch}': val_label_segments
    }

    test_segments_by_epoch[epoch] = {
        f'test_pred_segments_E{epoch}': test_pred_segments,
        f'test_label_segments_E{epoch}': test_label_segments
    }

    # 확인용 로그
    print(f"[Epoch {epoch}] Train seg shape: {train_pred_segments.shape}, Val: {val_pred_segments.shape}, Test: {test_pred_segments.shape}")


[Epoch ES] Train seg shape: torch.Size([4880, 65, 32, 1]), Val: torch.Size([665, 65, 32, 1]), Test: torch.Size([1422, 65, 32, 1])
[Epoch 1] Train seg shape: torch.Size([4880, 65, 32, 1]), Val: torch.Size([665, 65, 32, 1]), Test: torch.Size([1422, 65, 32, 1])


## Save 1st Performance Metrics, PV, and Segment

In [29]:
save_dir = './1st_results_ER'  # 저장할 디렉토리
os.makedirs(save_dir, exist_ok=True)

# 1st stage prediction 결과 저장
torch.save(train_1st_results, os.path.join(save_dir, 'train_1st_results.pt'))
torch.save(val_1st_results, os.path.join(save_dir, 'val_1st_results.pt'))
torch.save(test_1st_results, os.path.join(save_dir, 'test_1st_results.pt'))

# 세그먼트 결과 저장
torch.save(train_segments_by_epoch, os.path.join(save_dir, 'train_segments_by_epoch.pt'))
torch.save(val_segments_by_epoch, os.path.join(save_dir, 'val_segments_by_epoch.pt'))
torch.save(test_segments_by_epoch, os.path.join(save_dir, 'test_segments_by_epoch.pt'))

print("✅ All dictionaries saved successfully in ./saved_results")


✅ All dictionaries saved successfully in ./saved_results


# 2nd Model Training

In [30]:
train_segments_by_epoch = torch.load('./1st_results_ER/train_segments_by_epoch.pt', weights_only=False)
val_segments_by_epoch = torch.load('./1st_results_ER/val_segments_by_epoch.pt', weights_only=False)
test_segments_by_epoch = torch.load('./1st_results_ER/test_segments_by_epoch.pt', weights_only=False)

In [31]:

print(train_segments_by_epoch.keys())
print(train_segments_by_epoch['ES'].keys())
print(train_segments_by_epoch[1].keys())

dict_keys(['ES', 1])
dict_keys(['train_pred_segments_EES', 'train_label_segments_EES'])
dict_keys(['train_pred_segments_E1', 'train_label_segments_E1'])


In [32]:

train_pred_segments_ES = train_segments_by_epoch['ES']['train_pred_segments_EES']  # shape: (B, 65, segment_len, 1)
train_label_segments_ES = train_segments_by_epoch['ES']['train_label_segments_EES']
val_pred_segments_ES = val_segments_by_epoch['ES']['val_pred_segments_EES']  # shape: (B, 65, segment_len, 1)
val_label_segments_ES = val_segments_by_epoch['ES']['val_label_segments_EES']
test_pred_segments_ES = test_segments_by_epoch['ES']['test_pred_segments_EES']  # shape: (B, 65, segment_len, 1)
test_label_segments_ES = test_segments_by_epoch['ES']['test_label_segments_EES']


train_pred_segments_E1 = train_segments_by_epoch[1]['train_pred_segments_E1']  # shape: (B, 65, segment_len, 1)
train_label_segments_E1 = train_segments_by_epoch[1]['train_label_segments_E1']
val_pred_segments_E1 = val_segments_by_epoch[1]['val_pred_segments_E1']  # shape: (B, 65, segment_len, 1)
val_label_segments_E1 = val_segments_by_epoch[1]['val_label_segments_E1']
test_pred_segments_E1 = test_segments_by_epoch[1]['test_pred_segments_E1']  # shape: (B, 65, segment_len, 1)
test_label_segments_E1 = test_segments_by_epoch[1]['test_label_segments_E1']



## ES->1E

In [35]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path
import time


epo = 'ES'

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_pv):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          # 채널 수
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_pv    = attach_pv


# ── 공통 Config ───────────────────────────────────────────────
base_cfg = Configs(
    seq_len          = 336,
    pred_len         = 96,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = f"./model2_ER_E{epo}_1E",
    attach_pv  = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

loss_fn = nn.MSELoss()

# ── 세그먼트 루프 (65개) ──────────────────────────────────────
for seg in range(len(train_pred_segments_ES[0])):
    print(f"\n=========== Segment {seg} ===========")
    start_time = time.time()

    # GPU memory 사용량 초기화
    torch.cuda.reset_peak_memory_stats(device)
    mem_start = torch.cuda.memory_allocated(device)

    # ── 데이터로더 생성 (세그먼트 전용 pv) ───────────────
    tr_pv = train_pred_segments_ES[:, seg, :, :]   # (N,P,C)
    va_pv = val_pred_segments_ES[:,  seg, :, :]

    tr_ds = TensorDataset(train_sequences, train_labels, tr_pv)
    va_ds = TensorDataset(val_sequences,   val_labels,   va_pv)
    tr_dl = DataLoader(tr_ds, batch_size=8, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=8, shuffle=False)

    # ── 모델·채널별 옵티마이저·ES ───────────────────────
    model = Model(base_cfg).to(device)
    C     = model.channels                                    # =1

    opts = []
    for c in range(C):
        params = list(model.Linear[c].parameters())
        opts.append(optim.Adam(params, lr=base_cfg.learning_rate))
        path = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}"
        path.mkdir(parents=True, exist_ok=True)
        
    active = set(range(C))            # 현재 학습 중 채널 id

    # ── 학습 루프 (세그먼트별) ──────────────────────────
    for epoch in range(1, 2):
        if not active: break          # 모든 채널 종료 시 탈출

        # ‣ Train ------------------------------------------------d
        model.train()
        for c in active:                              # LR decay
            if base_cfg.lradj=='type1':
                lr = base_cfg.learning_rate * 0.5**((epoch-1)//1)
                for g in opts[c].param_groups: g['lr'] = lr

        printed_tensor_info = False  

        for xb, yb, pv in tr_dl:
            xb, yb, pv = xb.float().to(device), yb.float().to(device), pv.float().to(device)

            if not printed_tensor_info:
                # batch 당 tensor size 계산
                xb_size = xb.numel() * xb.element_size() / 1024**2
                yb_size = yb.numel() * yb.element_size() / 1024**2
                pv_size = pv.numel() * pv.element_size() / 1024**2

                # ─────────────────────────────
                # ① 전체 batch 개수 계산
                num_batches = len(tr_dl)

                # ② 전체 tensor 크기 계산
                total_xb_MB = num_batches * xb_size
                total_yb_MB = num_batches * yb_size
                total_pv_MB = num_batches * pv_size
                # ─────────────────────────────


                print(f"[Segment {seg}] Batch Size Info:")
                print(f"  xb per batch = {xb_size:.4f} MB, total = {total_xb_MB:.4f} MB")
                print(f"  yb per batch = {yb_size:.4f} MB, total = {total_yb_MB:.4f} MB")
                print(f"  pv per batch = {pv_size:.4f} MB, total = {total_pv_MB:.4f} MB")

                printed_tensor_info = True


            out = model(xb, pv)                       # fwd 1회

            loss_list = []
            for c in active:
                pred = out[:, -base_cfg.pred_len:, c]
                tgt  = yb[:, -base_cfg.pred_len:, c]
                loss_list.append(loss_fn(pred, tgt))

            total = torch.stack(loss_list).sum()
            for opt in opts:
                if opt: opt.zero_grad()
            total.backward()
            for c in active:
                opts[c].step()
        # ── 학습 시간 & 메모리 로깅 ─────────────────────────────
    elapsed = time.time() - start_time
    mem_end = torch.cuda.memory_allocated(device)
    peak_mem = torch.cuda.max_memory_allocated(device)


    print(f"[Segment {seg}] Training Time = {elapsed:.2f} sec")
    print(f"[Segment {seg}] GPU Memory Start = {mem_start/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Memory End   = {mem_end/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Peak Memory  = {peak_mem/1024**2:.2f} MB")

    print(f"\n============= Done =============")

    # ── seg 완료: 최종 state_dict(선택) 저장 --------------------
    torch.save(model.state_dict(),
                Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}" / "checkpoint.pth")
    del model, opts; torch.cuda.empty_cache()






=========== Segment 0 ===========
[Segment 0] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segment 0] Training Time = 1.97 sec
[Segment 0] GPU Memory Start = 196.38 MB
[Segment 0] GPU Memory End   = 196.94 MB
[Segment 0] GPU Peak Memory  = 197.96 MB

============= Done =============

=========== Segment 1 ===========
[Segment 1] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segment 1] Training Time = 1.27 sec
[Segment 1] GPU Memory Start = 196.94 MB
[Segment 1] GPU Memory End   = 197.08 MB
[Segment 1] GPU Peak Memory  = 198.10 MB

============= Done =============

=========== Segment 2 ===========
[Segment 2] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segme

## 1E->1E

In [36]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


epo = 1


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_pv):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          # 채널 수
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_pv    = attach_pv


# ── 공통 Config ───────────────────────────────────────────────
base_cfg = Configs(
    seq_len          = 336 ,
    pred_len         = 96,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = f"./model2_ER_E{epo}_1E/",
    attach_pv = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

loss_fn = nn.MSELoss()


# ── 세그먼트 루프 (65개) ──────────────────────────────────────
for seg in range(len(train_pred_segments_E1[0])):
    print(f"\n=========== Segment {seg} ===========")
    
    start_time = time.time()

    # GPU memory 사용량 초기화
    torch.cuda.reset_peak_memory_stats(device)
    mem_start = torch.cuda.memory_allocated(device)

    # ── 데이터로더 생성 (세그먼트 전용 pv) ───────────────
    tr_pv = train_pred_segments_E1[:, seg, :, :]   # (N,P,C)
    va_pv = val_pred_segments_E1[:,  seg, :, :]

    tr_ds = TensorDataset(train_sequences, train_labels, tr_pv)
    va_ds = TensorDataset(val_sequences,   val_labels,   va_pv)
    tr_dl = DataLoader(tr_ds, batch_size=8, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=8, shuffle=False)

    # ── 모델·채널별 옵티마이저·ES ───────────────────────
    model = Model(base_cfg).to(device)
    C     = model.channels                                    # =1

    opts = []
    for c in range(C):
        params = list(model.Linear[c].parameters())
        opts.append(optim.Adam(params, lr=base_cfg.learning_rate))
        path = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}"
        path.mkdir(parents=True, exist_ok=True)
        
    active = set(range(C))            # 현재 학습 중 채널 id

    # ── 학습 루프 (세그먼트별) ──────────────────────────
    for epoch in range(1, 2):
        if not active: break          # 모든 채널 종료 시 탈출

        # ‣ Train ------------------------------------------------d
        model.train()
        for c in active:                              # LR decay
            if base_cfg.lradj=='type1':
                lr = base_cfg.learning_rate * 0.5**((epoch-1)//1)
                for g in opts[c].param_groups: g['lr'] = lr

        printed_tensor_info = False
        
        for xb, yb, pv in tr_dl:
            xb, yb, pv = xb.float().to(device), yb.float().to(device), pv.float().to(device)

            if not printed_tensor_info:
                # batch 당 tensor size 계산
                xb_size = xb.numel() * xb.element_size() / 1024**2
                yb_size = yb.numel() * yb.element_size() / 1024**2
                pv_size = pv.numel() * pv.element_size() / 1024**2

                # ─────────────────────────────
                # ① 전체 batch 개수 계산
                num_batches = len(tr_dl)

                # ② 전체 tensor 크기 계산
                total_xb_MB = num_batches * xb_size
                total_yb_MB = num_batches * yb_size
                total_pv_MB = num_batches * pv_size
                # ─────────────────────────────


                print(f"[Segment {seg}] Batch Size Info:")
                print(f"  xb per batch = {xb_size:.4f} MB, total = {total_xb_MB:.4f} MB")
                print(f"  yb per batch = {yb_size:.4f} MB, total = {total_yb_MB:.4f} MB")
                print(f"  pv per batch = {pv_size:.4f} MB, total = {total_pv_MB:.4f} MB")

                printed_tensor_info = True



            out = model(xb, pv)                       # fwd 1회

            loss_list = []
            for c in active:
                pred = out[:, -base_cfg.pred_len:, c]
                tgt  = yb[:, -base_cfg.pred_len:, c]
                loss_list.append(loss_fn(pred, tgt))

            total = torch.stack(loss_list).sum()
            for opt in opts:
                if opt: opt.zero_grad()
            total.backward()
            for c in active:
                opts[c].step()

        # ── 학습 시간 & 메모리 로깅 ─────────────────────────────
    elapsed = time.time() - start_time
    mem_end = torch.cuda.memory_allocated(device)
    peak_mem = torch.cuda.max_memory_allocated(device)


    print(f"[Segment {seg}] Training Time = {elapsed:.2f} sec")
    print(f"[Segment {seg}] GPU Memory Start = {mem_start/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Memory End   = {mem_end/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Peak Memory  = {peak_mem/1024**2:.2f} MB")

    print(f"\n============= Done =============")

    # ── seg 완료: 최종 state_dict(선택) 저장 --------------------
    torch.save(model.state_dict(),
                Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}" / "checkpoint.pth")
    del model, opts; torch.cuda.empty_cache()



=========== Segment 0 ===========
[Segment 0] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segment 0] Training Time = 2.45 sec
[Segment 0] GPU Memory Start = 196.94 MB
[Segment 0] GPU Memory End   = 196.94 MB
[Segment 0] GPU Peak Memory  = 198.10 MB

============= Done =============

=========== Segment 1 ===========
[Segment 1] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segment 1] Training Time = 1.40 sec
[Segment 1] GPU Memory Start = 196.94 MB
[Segment 1] GPU Memory End   = 197.08 MB
[Segment 1] GPU Peak Memory  = 198.10 MB

============= Done =============

=========== Segment 2 ===========
[Segment 2] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segme

# 2nd Inference for Validation Set

In [37]:
train_segments_by_epoch = torch.load('./1st_results_ER/train_segments_by_epoch.pt', weights_only=False)
val_segments_by_epoch = torch.load('./1st_results_ER/val_segments_by_epoch.pt', weights_only=False)
test_segments_by_epoch = torch.load('./1st_results_ER/test_segments_by_epoch.pt', weights_only=False)

In [38]:
train_pred_segments_ES = train_segments_by_epoch['ES']['train_pred_segments_EES']  # shape: (B, 65, segment_len, 1)
train_label_segments_ES = train_segments_by_epoch['ES']['train_label_segments_EES']
val_pred_segments_ES = val_segments_by_epoch['ES']['val_pred_segments_EES']  # shape: (B, 65, segment_len, 1)
val_label_segments_ES = val_segments_by_epoch['ES']['val_label_segments_EES']
test_pred_segments_ES = test_segments_by_epoch['ES']['test_pred_segments_EES']  # shape: (B, 65, segment_len, 1)
test_label_segments_ES = test_segments_by_epoch['ES']['test_label_segments_EES']


train_pred_segments_E1 = train_segments_by_epoch[1]['train_pred_segments_E1']  # shape: (B, 65, segment_len, 1)
train_label_segments_E1 = train_segments_by_epoch[1]['train_label_segments_E1']
val_pred_segments_E1 = val_segments_by_epoch[1]['val_pred_segments_E1']  # shape: (B, 65, segment_len, 1)
val_label_segments_E1 = val_segments_by_epoch[1]['val_label_segments_E1']
test_pred_segments_E1 = test_segments_by_epoch[1]['test_pred_segments_E1']  # shape: (B, 65, segment_len, 1)
test_label_segments_E1 = test_segments_by_epoch[1]['test_label_segments_E1']


## ES->1E

### Configs

In [40]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_pv):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          # 채널 수
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_pv    = attach_pv


# ── 공통 Config ───────────────────────────────────────────────
base_cfg = Configs(
    pred_len         = 96,
    seq_len          = 336 ,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_EES_1E",
    attach_pv  = True,

)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [41]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path


C        = 1                                # 채널 수
P        = base_cfg.pred_len                # 96
seg_cnt  = 65
batch_sz = 8

results2_ER_ES_val_1E = {}  # 수정 x3   # {seg: { 'pred':(N,P,C), 'true':(N,P,C), 'metrics':{ch:(mse,mae,r2)} } }

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    # ── 세그먼트 전용 DataLoader (pv 포함) ─────────────────────
    pv = val_pred_segments_ES[:, seg, :, :]         # 수정         # (N,P,C)
    ds = TensorDataset(val_sequences, val_labels, pv) # 수정
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    # ── 예측 컨테이너 ────────────────────────────────────────
    pred_ch, true_ch, metrics = [], [], {}

    # ── 채널 루프 ────────────────────────────────────────────
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        # ① 모델 생성 & ckpt 로드
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 # xb:[B,S,C], yb:[B,P,C]
                xb  = xb.float().to(device)
                yb  = yb.float()      
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     # [B,P,C]
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        # ② 지표
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   # (N,P,C)
    true_all = torch.stack(true_ch,  dim=2)

    # 세그먼트별 평균 지표
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    # 저장(메모리)
    results2_ER_ES_val_1E[seg] = {
        "pred2_ES_val_1E"    : pred_all,   # 수정    # (N,P,C) tensor
        "true2_ES_val_1E"    : true_all,   # 수정
        "metrics2_ES_val_1E" : metrics     # 수정           # {ch:(mse,mae,r2)}
    }


seg10_ch0_mse = results2_ER_ES_val_1E[10]["metrics2_ES_val_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.316741406917572  MAE:0.43975862860679626
▶ seg0  mean  MSE:0.316741406917572  MAE:0.43975862860679626

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.23960784077644348  MAE:0.38832491636276245
▶ seg1  mean  MSE:0.23960784077644348  MAE:0.38832491636276245

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.24296164512634277  MAE:0.39656826853752136
▶ seg2  mean  MSE:0.24296164512634277  MAE:0.39656826853752136

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.44119352102279663  MAE:0.5201039910316467
▶ seg3  mean  MSE:0.44119352102279663  MAE:0.5201039910316467

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.2359120100736618  MAE:0.3806155323982239
▶ seg4  mean  MSE:0.2359120100736618  MAE:0.3806155323982239

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.21146160364151  MAE:0.35858890414237976
▶ seg5  mean  MSE:0.21146160364151  MAE:0.35858890414237976

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.250450134

## 1E->1E

### Configs

In [42]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_pv):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          # 채널 수
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_pv    = attach_pv


# ── 공통 Config ───────────────────────────────────────────────
base_cfg = Configs(
    pred_len         = 96,
    seq_len          = 336 ,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_E1_1E",
    attach_pv  = True,

)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [43]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path


C        = 1                                # 채널 수
P        = base_cfg.pred_len                # 96
seg_cnt  = 65
batch_sz = 8

results2_ER_E1_val_1E = {}  # 수정 x3   # {seg: { 'pred':(N,P,C), 'true':(N,P,C), 'metrics':{ch:(mse,mae,r2)} } }

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    # ── 세그먼트 전용 DataLoader (pv 포함) ─────────────────────
    pv = val_pred_segments_E1[:, seg, :, :]         # 수정         # (N,P,C)
    ds = TensorDataset(val_sequences, val_labels, pv) # 수정
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    # ── 예측 컨테이너 ────────────────────────────────────────
    pred_ch, true_ch, metrics = [], [], {}

    # ── 채널 루프 ────────────────────────────────────────────
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        # ① 모델 생성 & ckpt 로드
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 # xb:[B,S,C], yb:[B,P,C]
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     # [B,P,C]
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        # ② 지표
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    # ── (N,P,C) 텐서로 스택 ─────────────────────────────────
    pred_all = torch.stack(pred_ch,  dim=2)   # (N,P,C)
    true_all = torch.stack(true_ch,  dim=2)

    # 세그먼트별 평균 지표
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    # 저장(메모리)
    results2_ER_E1_val_1E[seg] = {
        "pred2_E1_val_1E"    : pred_all,   # 수정    # (N,P,C) tensor
        "true2_E1_val_1E"    : true_all,   # 수정
        "metrics2_E1_val_1E" : metrics     # 수정           # {ch:(mse,mae,r2)}
    }

# 예시: 세그먼트 10, 채널 3 의 MSE
seg10_ch0_mse = results2_ER_E1_val_1E[10]["metrics2_E1_val_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.31565213203430176  MAE:0.43641915917396545
▶ seg0  mean  MSE:0.31565213203430176  MAE:0.43641915917396545

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.2538796067237854  MAE:0.3942962884902954
▶ seg1  mean  MSE:0.2538796067237854  MAE:0.3942962884902954

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.21912875771522522  MAE:0.3621671199798584
▶ seg2  mean  MSE:0.21912875771522522  MAE:0.3621671199798584

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.42152124643325806  MAE:0.5196309089660645
▶ seg3  mean  MSE:0.42152124643325806  MAE:0.5196309089660645

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.2903937101364136  MAE:0.4313209354877472
▶ seg4  mean  MSE:0.2903937101364136  MAE:0.4313209354877472

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.2247549593448639  MAE:0.3736395835876465
▶ seg5  mean  MSE:0.2247549593448639  MAE:0.3736395835876465

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.243218317

# 2nd Inference for Test Set

## ES->1E

### Configs

In [44]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

pred_len = 96
seq_len = 336


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_pv):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          # 채널 수
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_pv    = attach_pv


# ── 공통 Config ───────────────────────────────────────────────
base_cfg = Configs(
    pred_len         = 96,
    seq_len          = 336,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_EES_1E",
    attach_pv  = True,
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [45]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path

C        = 1                                # 채널 수
P        = base_cfg.pred_len                # 96
seg_cnt  = 65
batch_sz = 8

results2_ER_ES_test_1E = {}  # 수정 x3   # {seg: { 'pred':(N,P,C), 'true':(N,P,C), 'metrics':{ch:(mse,mae,r2)} } }

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    # ── 세그먼트 전용 DataLoader (pv 포함) ─────────────────────
    pv = test_pred_segments_ES[:, seg, :, :]         # 수정         # (N,P,C)
    ds = TensorDataset(test_sequences, test_labels, pv) # 수정
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    # ── 예측 컨테이너 ────────────────────────────────────────
    pred_ch, true_ch, metrics = [], [], {}

    # ── 채널 루프 ────────────────────────────────────────────
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        # ① 모델 생성 & ckpt 로드
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 # xb:[B,S,C], yb:[B,P,C]
                xb  = xb.float().to(device)
                yb  = yb.float()        # CPU로 두어도 OK
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     # [B,P,C]
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     # (N,P)
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        # ② 지표
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    # ── (N,P,C) 텐서로 스택 ─────────────────────────────────
    pred_all = torch.stack(pred_ch,  dim=2)   # (N,P,C)
    true_all = torch.stack(true_ch,  dim=2)

    # 세그먼트별 평균 지표
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    # 저장(메모리)
    results2_ER_ES_test_1E[seg] = {
        "pred2_ES_test_1E"    : pred_all,   # 수정    # (N,P,C) tensor
        "true2_ES_test_1E"    : true_all,   # 수정
        "metrics2_ES_test_1E" : metrics     # 수정           # {ch:(mse,mae,r2)}
    }

# 예시: 세그먼트 10, 채널 3 의 MSE
seg10_ch0_mse = results2_ER_ES_test_1E[10]["metrics2_ES_test_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.16403837502002716  MAE:0.3170633018016815
▶ seg0  mean  MSE:0.16403837502002716  MAE:0.3170633018016815

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.12215931713581085  MAE:0.2742388844490051
▶ seg1  mean  MSE:0.12215931713581085  MAE:0.2742388844490051

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.12971220910549164  MAE:0.2902107238769531
▶ seg2  mean  MSE:0.12971220910549164  MAE:0.2902107238769531

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.33130884170532227  MAE:0.4619942009449005
▶ seg3  mean  MSE:0.33130884170532227  MAE:0.4619942009449005

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.11280112713575363  MAE:0.2577495276927948
▶ seg4  mean  MSE:0.11280112713575363  MAE:0.2577495276927948

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.12129656970500946  MAE:0.26184457540512085
▶ seg5  mean  MSE:0.12129656970500946  MAE:0.26184457540512085

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.120

## 1E->1E

### Configs

In [46]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

pred_len = 96
seq_len = 336 + pred_len//3


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_pv):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          # 채널 수
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_pv    = attach_pv


# ── 공통 Config ───────────────────────────────────────────────
base_cfg = Configs(
    pred_len         = 96,
    seq_len          = 336 ,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_E1_1E",
    attach_pv  = True,
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [47]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path

C        = 1                                # 채널 수
P        = base_cfg.pred_len                # 96
seg_cnt  = 65
batch_sz = 8

results2_ER_E1_test_1E = {}  # 수정 x3   # {seg: { 'pred':(N,P,C), 'true':(N,P,C), 'metrics':{ch:(mse,mae,r2)} } }

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    # ── 세그먼트 전용 DataLoader (pv 포함) ─────────────────────
    pv = test_pred_segments_E1[:, seg, :, :]         # 수정         # (N,P,C)
    ds = TensorDataset(test_sequences, test_labels, pv) # 수정
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    # ── 예측 컨테이너 ────────────────────────────────────────
    pred_ch, true_ch, metrics = [], [], {}

    # ── 채널 루프 ────────────────────────────────────────────
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        # ① 모델 생성 & ckpt 로드
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 # xb:[B,S,C], yb:[B,P,C]
                xb  = xb.float().to(device)
                yb  = yb.float()        # CPU로 두어도 OK
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     # [B,P,C]
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     # (N,P)
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        # ② 지표
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    # ── (N,P,C) 텐서로 스택 ─────────────────────────────────
    pred_all = torch.stack(pred_ch,  dim=2)   # (N,P,C)
    true_all = torch.stack(true_ch,  dim=2)

    # 세그먼트별 평균 지표
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    # 저장(메모리)
    results2_ER_E1_test_1E[seg] = {
        "pred2_E1_test_1E"    : pred_all,   # 수정    # (N,P,C) tensor
        "true2_E1_test_1E"    : true_all,   # 수정
        "metrics2_E1_test_1E" : metrics     # 수정           # {ch:(mse,mae,r2)}
    }

# 예시: 세그먼트 10, 채널 3 의 MSE
seg10_ch0_mse = results2_ER_E1_test_1E[10]["metrics2_E1_test_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.1615879386663437  MAE:0.3140833377838135
▶ seg0  mean  MSE:0.1615879386663437  MAE:0.3140833377838135

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.11811566352844238  MAE:0.2656703591346741
▶ seg1  mean  MSE:0.11811566352844238  MAE:0.2656703591346741

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.13090957701206207  MAE:0.27130448818206787
▶ seg2  mean  MSE:0.13090957701206207  MAE:0.27130448818206787

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.2727861702442169  MAE:0.4301261901855469
▶ seg3  mean  MSE:0.2727861702442169  MAE:0.4301261901855469

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.17446111142635345  MAE:0.3406302034854889
▶ seg4  mean  MSE:0.17446111142635345  MAE:0.3406302034854889

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.11139986664056778  MAE:0.2553647756576538
▶ seg5  mean  MSE:0.11139986664056778  MAE:0.2553647756576538

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.1188780

# Rank Models (by 2nd validation performance)

In [48]:
results1_ER_train = torch.load("./1st_results_ER/train_1st_results.pt")
results1_ER_val = torch.load("./1st_results_ER/val_1st_results.pt")
results1_ER_test = torch.load("./1st_results_ER/test_1st_results.pt")

### ES->1E

In [49]:

import numpy as np
from sklearn.metrics import mean_squared_error

# ──────────────────────────────────────────────
# 0. 설정
# ──────────────────────────────────────────────
TOP_N       = 65
SEG_CNT     = 65
CH_CNT      = 1
metric_idx  = 0          # (mse, mae, r2) 중 0 = MSE

# ──────────────────────────────────────────────
# 1. 증강 모델 MSE 테이블 수집 → (SEG_CNT, CH_CNT)
# ──────────────────────────────────────────────
mse_table = np.full((SEG_CNT, CH_CNT), np.inf)

for seg in range(SEG_CNT):
    if seg not in results2_ER_ES_val_1E:
        continue
    for ch in range(CH_CNT):
        if ch in results2_ER_ES_val_1E[seg]["metrics2_ES_val_1E"]:
            mse_table[seg, ch] = results2_ER_ES_val_1E[seg]["metrics2_ES_val_1E"][ch][metric_idx]

# ──────────────────────────────────────────────
# 2. 채널별 Top-N 인덱스 추출 & 저장
# ──────────────────────────────────────────────
print("\n▶ Train-set  Per-Channel  Top-N  MSE\n")
top_seg_idx_ES_val_1E = {}                       # ← 여기 저장

for ch in range(CH_CNT):
    print(f"=== Channel {ch} ===")
    v_mse = results1_ER_val[1]['mse_val_E1']
    print(f"Vanilla MSE : {v_mse:.6f}\n")

    # (seg, mse) 중 유효 값만 골라 MSE 오름차순
    pairs = [(seg, mse_table[seg, ch]) for seg in range(SEG_CNT)
             if np.isfinite(mse_table[seg, ch])]
    top   = sorted(pairs, key=lambda x: x[1])[:TOP_N]

    # ── 인덱스만 따로 저장 ─────────────────────
    top_seg_idx_ES_val_1E[ch] = [seg for seg, _ in top]   # e.g. {0:[12,5,7,…], 1:[3,8,…]}

    # ── 출력 ─────────────────────────────────
    for rank, (seg_id, mse_val) in enumerate(top, 1):
        delta = v_mse - mse_val
        sign  = "+" if delta > 0 else "-"
        print(f"  Top {rank:2d} | seg {seg_id:02d} | MSE {mse_val:.6f} "
              f"({sign}{abs(delta):.6f})")
    print()

# ──────────────────────────────────────────────
# 3. 확인
# ──────────────────────────────────────────────
print("Saved top segment indices per channel:")
for ch, seg_list in top_seg_idx_ES_val_1E.items():
    print(f"Channel {ch}: {seg_list}")



▶ Train-set  Per-Channel  Top-N  MSE

=== Channel 0 ===
Vanilla MSE : 0.217507

  Top  1 | seg 05 | MSE 0.211462 (+0.006046)
  Top  2 | seg 29 | MSE 0.216269 (+0.001238)
  Top  3 | seg 53 | MSE 0.218803 (-0.001296)
  Top  4 | seg 24 | MSE 0.218826 (-0.001318)
  Top  5 | seg 57 | MSE 0.221429 (-0.003921)
  Top  6 | seg 42 | MSE 0.222881 (-0.005373)
  Top  7 | seg 36 | MSE 0.222911 (-0.005403)
  Top  8 | seg 19 | MSE 0.223763 (-0.006255)
  Top  9 | seg 39 | MSE 0.224822 (-0.007315)
  Top 10 | seg 14 | MSE 0.225524 (-0.008016)
  Top 11 | seg 47 | MSE 0.230050 (-0.012542)
  Top 12 | seg 48 | MSE 0.231416 (-0.013908)
  Top 13 | seg 46 | MSE 0.231695 (-0.014188)
  Top 14 | seg 21 | MSE 0.234102 (-0.016594)
  Top 15 | seg 61 | MSE 0.234487 (-0.016980)
  Top 16 | seg 23 | MSE 0.235043 (-0.017535)
  Top 17 | seg 04 | MSE 0.235912 (-0.018405)
  Top 18 | seg 07 | MSE 0.236777 (-0.019270)
  Top 19 | seg 55 | MSE 0.238088 (-0.020580)
  Top 20 | seg 56 | MSE 0.239110 (-0.021602)
  Top 21 | seg 01 |

### 1E->1E

In [50]:

import numpy as np
from sklearn.metrics import mean_squared_error

# ──────────────────────────────────────────────
# 0. 설정
# ──────────────────────────────────────────────
TOP_N       = 65
SEG_CNT     = 65
CH_CNT      = 1
metric_idx  = 0          # (mse, mae, r2) 중 0 = MSE

# ──────────────────────────────────────────────
# 1. 증강 모델 MSE 테이블 수집 → (SEG_CNT, CH_CNT)
# ──────────────────────────────────────────────
mse_table = np.full((SEG_CNT, CH_CNT), np.inf)

for seg in range(SEG_CNT):
    if seg not in results2_ER_E1_val_1E:
        continue
    for ch in range(CH_CNT):
        if ch in results2_ER_E1_val_1E[seg]["metrics2_E1_val_1E"]:
            mse_table[seg, ch] = results2_ER_E1_val_1E[seg]["metrics2_E1_val_1E"][ch][metric_idx]

# ──────────────────────────────────────────────
# 2. 채널별 Top-N 인덱스 추출 & 저장
# ──────────────────────────────────────────────
print("\n▶ Train-set  Per-Channel  Top-N  MSE\n")
top_seg_idx_E1_val_1E = {}                       # ← 여기 저장

for ch in range(CH_CNT):
    print(f"=== Channel {ch} ===")
    v_mse = results1_ER_val[1]['mse_val_E1']
    print(f"Vanilla MSE : {v_mse:.6f}\n")

    # (seg, mse) 중 유효 값만 골라 MSE 오름차순
    pairs = [(seg, mse_table[seg, ch]) for seg in range(SEG_CNT)
             if np.isfinite(mse_table[seg, ch])]
    top   = sorted(pairs, key=lambda x: x[1])[:TOP_N]

    # ── 인덱스만 따로 저장 ─────────────────────
    top_seg_idx_E1_val_1E[ch] = [seg for seg, _ in top]   # e.g. {0:[12,5,7,…], 1:[3,8,…]}

    # ── 출력 ─────────────────────────────────
    for rank, (seg_id, mse_val) in enumerate(top, 1):
        delta = v_mse - mse_val
        sign  = "+" if delta > 0 else "-"
        print(f"  Top {rank:2d} | seg {seg_id:02d} | MSE {mse_val:.6f} "
              f"({sign}{abs(delta):.6f})")
    print()

# ──────────────────────────────────────────────
# 3. 확인
# ──────────────────────────────────────────────
print("Saved top segment indices per channel:")
for ch, seg_list in top_seg_idx_E1_val_1E.items():
    print(f"Channel {ch}: {seg_list}")



▶ Train-set  Per-Channel  Top-N  MSE

=== Channel 0 ===
Vanilla MSE : 0.217507

  Top  1 | seg 61 | MSE 0.216221 (+0.001286)
  Top  2 | seg 53 | MSE 0.218959 (-0.001451)
  Top  3 | seg 02 | MSE 0.219129 (-0.001621)
  Top  4 | seg 63 | MSE 0.219528 (-0.002021)
  Top  5 | seg 42 | MSE 0.220670 (-0.003163)
  Top  6 | seg 45 | MSE 0.220875 (-0.003368)
  Top  7 | seg 56 | MSE 0.221263 (-0.003755)
  Top  8 | seg 46 | MSE 0.222764 (-0.005257)
  Top  9 | seg 40 | MSE 0.222912 (-0.005404)
  Top 10 | seg 21 | MSE 0.222979 (-0.005472)
  Top 11 | seg 47 | MSE 0.223130 (-0.005622)
  Top 12 | seg 33 | MSE 0.223336 (-0.005828)
  Top 13 | seg 58 | MSE 0.224638 (-0.007131)
  Top 14 | seg 05 | MSE 0.224755 (-0.007247)
  Top 15 | seg 25 | MSE 0.225851 (-0.008343)
  Top 16 | seg 28 | MSE 0.226484 (-0.008976)
  Top 17 | seg 41 | MSE 0.226782 (-0.009274)
  Top 18 | seg 64 | MSE 0.228383 (-0.010875)
  Top 19 | seg 34 | MSE 0.230000 (-0.012493)
  Top 20 | seg 44 | MSE 0.233962 (-0.016454)
  Top 21 | seg 43 |

# Ensemble

## ES->1E


In [51]:
import torch, os, numpy as np
from torch.utils.data import DataLoader, TensorDataset

TOP_K    = 65
CH_CNT   = 1
batch_sz = 8

def infer_one(model, loader, ch):
    buf = []
    model.eval()
    with torch.no_grad():
        for xb, yb, pv in loader:
            xb, pv = xb.to(device), pv.to(device)
            buf.append(model(xb, pv)[:, :, ch].cpu())   # [B,P]
    return torch.cat(buf, 0)   # [N,P]


van_mse_list, ens_mse_list, delta_mse_list, delta_mse_pct_list = [], [], [], []
van_mae_list, ens_mae_list, delta_mae_list, delta_mae_pct_list = [], [], [], []

# ─────────────────────────────────────────────
# 채널 루프
# ─────────────────────────────────────────────
for ch in range(CH_CNT):
    print(f"\n=== Channel {ch} ===")

    # -----------------------------
    # 1) Vanilla MSE & MAE
    # -----------------------------
    v_mse = results1_ER_test['ES']['mse_test_EES']
    v_mae = results1_ER_test['ES']['mae_test_EES']

    print(f"[Vanilla]   MSE : {v_mse:.6f} | MAE : {v_mae:.6f}")

    van_mse_list.append(v_mse)
    van_mae_list.append(v_mae)

    preds_test_arr = []
    ok_segs = []

    # ─────────────── Top-K 모델 로드 및 예측 ───────────────
    for seg in top_seg_idx_ES_val_1E[ch][:TOP_K]:
        ckpt = f"./model2_ER_EES_1E/seg{seg}/ch{ch}/checkpoint.pth"
        if not os.path.exists(ckpt):
            print(f"  [skip] ckpt missing : seg{seg}")
            continue

        pv_seg = test_pred_segments_ES[:, seg, :, :]
        dl     = DataLoader(TensorDataset(test_sequences, test_labels, pv_seg),
                            batch_size=batch_sz, shuffle=False)

        mdl = Model(base_cfg).to(device)
        mdl.load_state_dict(torch.load(ckpt, map_location=device))

        preds_test_arr.append(infer_one(mdl, dl, ch))   # [N,P]
        ok_segs.append(seg)

    if len(preds_test_arr) < 5:
        print("  Not enough valid models.")
        continue

    # ─────────────────────────────────────────────
    # Var / Corr 계산
    # ─────────────────────────────────────────────
    step = 5
    Ks = list(range(step, len(preds_test_arr)+1, step))

    var_list, rho_list = [], []

    for K in Ks:
        F = torch.stack(preds_test_arr[:K], dim=0)

        V = torch.var(F, dim=0).mean().item()
        var_list.append(V)

        flat = F.reshape(K, -1).numpy()
        corr = np.corrcoef(flat)
        R = np.mean(np.abs(corr[np.triu_indices(K, 1)]))
        rho_list.append(R)

    # 정규화
    V_min, V_max = min(var_list), max(var_list)
    R_min, R_max = min(rho_list), max(rho_list)

    V_norm = [(v - V_min)/(V_max - V_min + 1e-12) for v in var_list]
    R_norm = [(r - R_min)/(R_max - R_min + 1e-12) for r in rho_list]

    alpha, beta = 1, 1
    scores = [alpha*v + beta*r for v, r in zip(V_norm, R_norm)]

    best_K = Ks[int(np.argmin(scores))]

    print("  — Var / |Corr| / Score —")
    for k, v, r, s in zip(Ks, var_list, rho_list, scores):
        tag = "<- pick" if k == best_K else ""
        print(f"   K={k:2d} | Var={v:.3e} | |ρ|={r:.4f} | S={s:.4f} {tag}")

    # ─────────────────────────────────────────────
    # Ensemble 결과 계산
    # ─────────────────────────────────────────────
    ens_pred = torch.mean(torch.stack(preds_test_arr[:best_K]), dim=0)
    y_true   = test_labels[:, -base_cfg.pred_len:, ch].cpu()

    mse_ens = torch.mean((ens_pred - y_true)**2).item()
    mae_ens = torch.mean(torch.abs(ens_pred - y_true)).item()

    ens_mse_list.append(mse_ens)
    ens_mae_list.append(mae_ens)

    # Δ (절대 변화량)
    delta_mse = v_mse - mse_ens
    delta_mae = v_mae - mae_ens

    # Δ% (퍼센트 변화량)
    delta_mse_pct = (delta_mse / v_mse) * 100
    delta_mae_pct = (delta_mae / v_mae) * 100

    delta_mse_list.append(delta_mse)
    delta_mae_list.append(delta_mae)

    delta_mse_pct_list.append(delta_mse_pct)
    delta_mae_pct_list.append(delta_mae_pct)

    print(f"[Var-Corr]  K={best_K}")
    print(f"  MSE = {mse_ens:.6f} | Δ = {delta_mse:+.6f} | Δ% = {delta_mse_pct:+.2f}%")
    print(f"  MAE = {mae_ens:.6f} | Δ = {delta_mae:+.6f} | Δ% = {delta_mae_pct:+.2f}%")

# ─────────────────────────────────────────────
# Summary
# ─────────────────────────────────────────────
if ens_mse_list:
    print("\n──────── Summary (Var-Corr) ───────")
    print(f"Vanilla  mean MSE : {np.mean(van_mse_list):.6f}")
    print(f"Ensemble mean MSE : {np.mean(ens_mse_list):.6f}")
    print(f"Average Δ MSE     : {np.mean(delta_mse_list):+.6f}")
    print(f"Average Δ MSE (%) : {np.mean(delta_mse_pct_list):+.2f}%\n")

    print(f"Vanilla  mean MAE : {np.mean(van_mae_list):.6f}")
    print(f"Ensemble mean MAE : {np.mean(ens_mae_list):.6f}")
    print(f"Average Δ MAE     : {np.mean(delta_mae_list):+.6f}")
    print(f"Average Δ MAE (%) : {np.mean(delta_mae_pct_list):+.2f}%")



=== Channel 0 ===
[Vanilla]   MSE : 0.111791 | MAE : 0.255528
  — Var / |Corr| / Score —
   K= 5 | Var=1.253e-02 | |ρ|=0.9807 | S=0.9577 
   K=10 | Var=1.466e-02 | |ρ|=0.9813 | S=1.0926 
   K=15 | Var=1.721e-02 | |ρ|=0.9774 | S=0.9249 <- pick
   K=20 | Var=1.740e-02 | |ρ|=0.9775 | S=0.9427 
   K=25 | Var=1.851e-02 | |ρ|=0.9775 | S=0.9900 
   K=30 | Var=1.839e-02 | |ρ|=0.9776 | S=0.9910 
   K=35 | Var=1.849e-02 | |ρ|=0.9769 | S=0.9465 
   K=40 | Var=1.947e-02 | |ρ|=0.9761 | S=0.9356 
   K=45 | Var=2.089e-02 | |ρ|=0.9757 | S=0.9694 
   K=50 | Var=2.181e-02 | |ρ|=0.9749 | S=0.9480 
   K=55 | Var=2.388e-02 | |ρ|=0.9739 | S=0.9712 
   K=60 | Var=2.759e-02 | |ρ|=0.9725 | S=1.0295 
   K=65 | Var=3.548e-02 | |ρ|=0.9672 | S=1.0000 
[Var-Corr]  K=15
  MSE = 0.104257 | Δ = +0.007533 | Δ% = +6.74%
  MAE = 0.245495 | Δ = +0.010033 | Δ% = +3.93%

──────── Summary (Var-Corr) ───────
Vanilla  mean MSE : 0.111791
Ensemble mean MSE : 0.104257
Average Δ MSE     : +0.007533
Average Δ MSE (%) : +6.74%

Va

## 1E->1E


In [52]:
results1_ER_test['ES']['mae_test_EES']

0.2555276155471802

In [53]:
import torch, os, numpy as np
from torch.utils.data import DataLoader, TensorDataset

TOP_K    = 65
CH_CNT   = 1
batch_sz = 8

def infer_one(model, loader, ch):
    buf = []
    model.eval()
    with torch.no_grad():
        for xb, yb, pv in loader:
            xb, pv = xb.to(device), pv.to(device)
            buf.append(model(xb, pv)[:, :, ch].cpu())   # [B,P]
    return torch.cat(buf, 0)    # [N,P]


# 저장 리스트
van_mse_list, ens_mse_list, delta_mse_list, delta_mse_pct_list = [], [], [], []
van_mae_list, ens_mae_list, delta_mae_list, delta_mae_pct_list = [], [], [], []


# ─────────────────────────────────────────────
# 채널 루프
# ─────────────────────────────────────────────
for ch in range(CH_CNT):

    print(f"\n=== Channel {ch} ===")

    # 1) Vanilla Scores
    v_mse = results1_ER_test['ES']['mse_test_EES']
    v_mae = results1_ER_test['ES']['mae_test_EES']

    print(f"[Vanilla]   MSE : {v_mse:.6f} | MAE : {v_mae:.6f}")

    van_mse_list.append(v_mse)
    van_mae_list.append(v_mae)

    preds_test_arr = []
    ok_segs = []

    # ─────────────────────────────────────────────
    # Top-K 모델 로드
    # ─────────────────────────────────────────────
    for seg in top_seg_idx_E1_val_1E[ch][:TOP_K]:

        ckpt = f"./model2_ER_E1_1E/seg{seg}/ch{ch}/checkpoint.pth"
        if not os.path.exists(ckpt):
            print(f"  [skip] missing checkpoint : seg{seg}")
            continue

        mdl = Model(base_cfg).to(device)
        mdl.load_state_dict(torch.load(ckpt, map_location=device))

        pv_seg = test_pred_segments_E1[:, seg, :, :]
        dl = DataLoader(
            TensorDataset(test_sequences, test_labels, pv_seg),
            batch_size=batch_sz, shuffle=False
        )

        preds_test_arr.append(infer_one(mdl, dl, ch))
        ok_segs.append(seg)

    if len(preds_test_arr) < 5:
        print("  Not enough valid models.")
        continue

    # ─────────────────────────────────────────────
    # K 후보 (예: 5, 10, 15...)
    # ─────────────────────────────────────────────
    step = 5
    Ks = list(range(step, len(preds_test_arr)+1, step))

    var_list = []
    rho_list = []

    for K in Ks:
        F = torch.stack(preds_test_arr[:K], dim=0)   # [K,N,P]

        # 분산
        V = torch.var(F, dim=0).mean().item()
        var_list.append(V)

        # 상관
        flat = F.reshape(K, -1).numpy()
        corr = np.corrcoef(flat)
        R    = np.mean(np.abs(corr[np.triu_indices(K, 1)]))
        rho_list.append(R)

    # ─────────────────────────────────────────────
    # 정규화
    # ─────────────────────────────────────────────
    V_min, V_max = min(var_list), max(var_list)
    R_min, R_max = min(rho_list), max(rho_list)

    V_norm = [(v - V_min) / (V_max - V_min + 1e-12) for v in var_list]
    R_norm = [(r - R_min) / (R_max - R_min + 1e-12) for r in rho_list]

    alpha, beta = 1, 1.1   # weight

    scores = [alpha*v + beta*r for v, r in zip(V_norm, R_norm)]
    best_K = Ks[int(np.argmin(scores))]

    # ─────────────────────────────────────────────
    # K Table 출력
    # ─────────────────────────────────────────────
    print("  — Var / |Corr| / Score —")
    for k, v, r, s in zip(Ks, var_list, rho_list, scores):
        tag = "<- pick" if k == best_K else ""
        print(f"   K={k:2d} | Var={v:.3e} | |ρ|={r:.4f} | S={s:.4f} {tag}")

    # ─────────────────────────────────────────────
    # 최종 Test Ensemble
    # ─────────────────────────────────────────────
    ens_pred = torch.mean(torch.stack(preds_test_arr[:best_K]), dim=0)  # [N,P]
    y_true   = test_labels[:, -base_cfg.pred_len:, ch].cpu()

    mse_ens = torch.mean((ens_pred - y_true)**2).item()
    mae_ens = torch.mean(torch.abs(ens_pred - y_true)).item()

    ens_mse_list.append(mse_ens)
    ens_mae_list.append(mae_ens)

    # Δ absolute
    delta_mse = v_mse - mse_ens
    delta_mae = v_mae - mae_ens

    # Δ percent
    delta_mse_pct = (delta_mse / v_mse) * 100
    delta_mae_pct = (delta_mae / v_mae) * 100

    delta_mse_list.append(delta_mse)
    delta_mae_list.append(delta_mae)
    delta_mse_pct_list.append(delta_mse_pct)
    delta_mae_pct_list.append(delta_mae_pct)

    sign = "+" if delta_mse > 0 else "-"
    print(f"[Var-Corr]  K={best_K}")
    print(f"   MSE = {mse_ens:.6f} | Δ = {delta_mse:+.6f} | Δ% = {delta_mse_pct:+.2f}%")
    print(f"   MAE = {mae_ens:.6f} | Δ = {delta_mae:+.6f} | Δ% = {delta_mae_pct:+.2f}%")


# ─────────────────────────────────────────────
# Summary
# ─────────────────────────────────────────────
if ens_mse_list:
    print("\n──────── Summary (Var-Corr) ───────")
    print(f"Vanilla  mean MSE  : {np.mean(van_mse_list):.6f}")
    print(f"Ensemble mean MSE  : {np.mean(ens_mse_list):.6f}")
    print(f"Average Δ MSE      : {np.mean(delta_mse_list):+.6f}")
    print(f"Average Δ MSE (%)  : {np.mean(delta_mse_pct_list):+.2f}%\n")

    print(f"Vanilla  mean MAE  : {np.mean(van_mae_list):.6f}")
    print(f"Ensemble mean MAE  : {np.mean(ens_mae_list):.6f}")
    print(f"Average Δ MAE      : {np.mean(delta_mae_list):+.6f}")
    print(f"Average Δ MAE (%)  : {np.mean(delta_mae_pct_list):+.2f}%")



=== Channel 0 ===
[Vanilla]   MSE : 0.111791 | MAE : 0.255528
  — Var / |Corr| / Score —
   K= 5 | Var=1.413e-02 | |ρ|=0.9855 | S=1.1100 
   K=10 | Var=1.379e-02 | |ρ|=0.9850 | S=1.0690 
   K=15 | Var=1.458e-02 | |ρ|=0.9840 | S=1.0350 
   K=20 | Var=1.618e-02 | |ρ|=0.9839 | S=1.0796 
   K=25 | Var=1.904e-02 | |ρ|=0.9828 | S=1.0998 
   K=30 | Var=2.141e-02 | |ρ|=0.9791 | S=0.9542 
   K=35 | Var=2.361e-02 | |ρ|=0.9784 | S=0.9761 
   K=40 | Var=2.537e-02 | |ρ|=0.9773 | S=0.9656 
   K=45 | Var=2.832e-02 | |ρ|=0.9745 | S=0.8885 <- pick
   K=50 | Var=3.193e-02 | |ρ|=0.9730 | S=0.9070 
   K=55 | Var=3.586e-02 | |ρ|=0.9719 | S=0.9605 
   K=60 | Var=4.062e-02 | |ρ|=0.9693 | S=0.9513 
   K=65 | Var=4.715e-02 | |ρ|=0.9668 | S=1.0000 
[Var-Corr]  K=45
   MSE = 0.102114 | Δ = +0.009677 | Δ% = +8.66%
   MAE = 0.245830 | Δ = +0.009697 | Δ% = +3.79%

──────── Summary (Var-Corr) ───────
Vanilla  mean MSE  : 0.111791
Ensemble mean MSE  : 0.102114
Average Δ MSE      : +0.009677
Average Δ MSE (%)  : +8.6